In [1]:
# @title marker-pdf 및 필수 의존성 설치
from google.colab import output

!pip install marker-pdf torch transformers

# 출력 지우기
output.clear()
print("설치 완료")

설치 완료


In [3]:
# @title PDF to Markdown+Image ZIP

import os
import shutil
import torch
from google.colab import files
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered
from marker.config.parser import ConfigParser

# 1. 커스텀 저장 함수 정의 (Pillow 오류 방지용)
def safe_save_output(rendered, output_dir, base_name):
    """
    marker의 기본 save_output 대신 사용하는 안전한 저장 함수입니다.
    JPEG 대신 PNG를 사용하여 인코더 충돌 오류를 방지합니다.
    """
    # 텍스트 및 메타데이터 추출
    text, _, _ = text_from_rendered(rendered)

    # 마크다운 파일 저장
    md_filename = f"{base_name}.md"
    md_path = os.path.join(output_dir, md_filename)
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(text)

    # 이미지 저장 (오류가 발생하는 JPEG 대신 PNG 사용)
    # rendered.images는 {파일명: PIL_Image_Object} 형태의 딕셔너리입니다.
    if rendered.images:
        for img_filename, img_obj in rendered.images.items():
            # 이미지 파일명에서 확장자가 없을 경우 안전하게 처리
            if not os.path.splitext(img_filename)[1]:
                img_filename += ".png"

            # 원본이 JPEG라도 PNG로 저장하여 호환성 확보
            # (PIL C-API 충돌 회피)
            save_path = os.path.join(output_dir, img_filename)
            try:
                img_obj.save(save_path, "PNG")
            except Exception as img_e:
                print(f"[경고] 이미지 저장 실패 ({img_filename}): {img_e}")

    return md_path

# ---------------------------------------------------------
# 2. 시스템 설정 및 모델 로드
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[시스템] 연산 장치: {device}")

# 모델 로드
print("[시스템] 모델 및 컨버터 초기화 중...")
model_dict = create_model_dict()

# 설정: 이미지 추출 활성화
config_parser = ConfigParser({
    "output_format": "markdown",
    "extract_images": True,
    "image_layer_handling": "merge"
})

converter = PdfConverter(
    artifact_dict=model_dict,
    config=config_parser.generate_config_dict(),
    processor_list=config_parser.get_processors(),
    renderer=config_parser.get_renderer()
)
print("[시스템] 준비 완료.")

# ---------------------------------------------------------
# 3. 파일 업로드 및 변환
# ---------------------------------------------------------
print("\n[입력] 변환할 PDF 파일을 업로드하세요 (개별 ZIP으로 생성됩니다).")
uploaded = files.upload()

# 임시 작업 공간
base_work_dir = os.path.abspath("./temp_processing")

for filename in uploaded.keys():
    file_path = os.path.abspath(filename)
    base_name = os.path.splitext(filename)[0]

    # 작업 디렉토리: ./temp_processing/{파일명}
    work_dir = os.path.join(base_work_dir, base_name)

    # 기존 폴더 정리 및 생성
    if os.path.exists(work_dir):
        shutil.rmtree(work_dir)
    os.makedirs(work_dir, exist_ok=True)

    print(f"\n[처리중] {filename} 변환 시작...")

    try:
        # 변환 실행
        rendered = converter(file_path)

        # [핵심 변경] 커스텀 저장 함수 사용
        safe_save_output(rendered, work_dir, base_name)

        # ZIP 압축
        zip_filename = f"{base_name}.zip"
        shutil.make_archive(base_name, 'zip', work_dir)

        print(f"[완료] {zip_filename} 생성 완료.")
        files.download(zip_filename)

    except Exception as e:
        print(f"[오류] {filename} 변환 중 치명적 오류: {e}")
        import traceback
        traceback.print_exc()

    finally:
        # 디스크 공간 확보를 위해 작업 폴더 삭제 (선택사항)
        # shutil.rmtree(work_dir, ignore_errors=True)
        pass

print("\n[시스템] 모든 작업이 종료되었습니다.")

[시스템] 연산 장치: cuda
[시스템] 모델 및 컨버터 초기화 중...
[시스템] 준비 완료.

[입력] 변환할 PDF 파일을 업로드하세요 (개별 ZIP으로 생성됩니다).


Saving 2405.02100v2.pdf to 2405.02100v2 (1).pdf

[처리중] 2405.02100v2 (1).pdf 변환 시작...


Recognizing Text: 100%|██████████| 41/41 [00:19<00:00,  2.12it/s]
Detecting bboxes: 0it [00:00, ?it/s]


[완료] 2405.02100v2 (1).zip 생성 완료.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


[시스템] 모든 작업이 종료되었습니다.
